In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sporgboost.preprocessing import onehot_encode, shuffle
from sporgboost.forests._base import BaseRandomForest, BaseAdaBoost
from sklearn.metrics import roc_auc_score
model_rf = BaseRandomForest()
model_gb = BaseAdaBoost()

In [2]:
# Dataset for testing
X, y = load_iris(return_X_y = True)

# Set seed to be able to reproduce
np.random.seed(1234)
    
# Preprocessing
y = onehot_encode(y)
X, y = shuffle(X, y)

# Train and Test Splits
X_train, y_train = X[:-50,:], y[:-50,:]
X_test, y_test = X[-50:,:], y[-50:,:]


In [3]:
%timeit model_rf.fit(X_train, y_train)

285 ms ± 4.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit model_gb.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
roc_auc_score(y_test, model_rf.predict_proba(X_test), multi_class='ovo')

0.998799148408656

In [7]:
roc_auc_score(y_test, model_gb.predict_proba(X_test), multi_class='ovo')

0.9791300832726978

In [7]:
model_gb.n_trees

3